In [1]:
# Include all the utilities
include("utilities.jl");

In [19]:
# Read mazes that should be solved
max_size = 10
n_averages = 100

mazes = [zeros(2*i+1, 2*i+1) for i in 1:max_size, j in 1:n_averages];

for i in 1:max_size
    test = readdlm("mazes/mazes$i.csv", ',', Float64)
    test = reshape(test, 100, 2*i+1, 2*i+1)
    for j in 1:n_averages
        mazes[i,j] = test[j,:,:]
    end
end

In [20]:
# Allocate memory for the computations with fixed discount
timesBCP = zeros(max_size, n_averages);
rewardsBCP = zeros(max_size, n_averages);
statusBCP = [];

In [21]:
# Define the visibility / observability mask
V = [1 1 1; 1 0 1; 1 1 1];

In [22]:
# Fix discount factor
γ = 0.9999;

In [23]:
# Do the computations
@elapsed for i in 1:max_size
    for j in 1:n_averages
        M = mazes[i,j];
        states = listOfStates(M);
        goal = rand(states);
        α = transitionKernel(M,A,goal);
        β = observationKernel(M,V);
        r = instReward(M,A,goal);
        μ = initialDistribution(M);
        # Evaluate BCP
        πBCP, modelBCP = BCP(α, β, r, μ, γ, 0)
        rewardsBCP[i,j] = RExact(πBCP, α, β, γ, μ, r)
        timesBCP[i,j] = solve_time(modelBCP)
        statusBCP = vcat(statusBCP, raw_status(modelBCP))
    end
    println("BCP time for size $i: ", sum(timesBCP[i,:]) / n_averages)
    println("BCP reward for size $i: ", sum(rewardsBCP[i,:]) / n_averages)
end

BCP time for size 1: 0.004149513244628906
BCP reward for size 1: 0.9999999999999999
BCP time for size 2: 0.012756757736206055
BCP reward for size 2: 2.144767199619089
BCP time for size 3: 0.030941081047058106
BCP reward for size 3: 2.444624776910715
BCP time for size 4: 0.18306411266326905
BCP reward for size 4: 2.0875416817698307
BCP time for size 5: 0.5015405488014221
BCP reward for size 5: 1.2728697003485945
BCP time for size 6: 1.8053146076202393
BCP reward for size 6: 0.5399709252046625
BCP time for size 7: 2.136158497333527
BCP reward for size 7: 2.0923987838754003
BCP time for size 8: 3.2971184039115906
BCP reward for size 8: 0.3839107297748411
BCP time for size 9: 5.861895041465759
BCP reward for size 9: 0.314351113039053
BCP time for size 10: 7.07161171913147
BCP reward for size 10: 0.27354067202295673


2139.700251571

In [24]:
writedlm("data/BCPDiscountFixedRewardsSmall.csv", rewardsBCP, ',')
writedlm("data/BCPDiscountFixedTimesSmall.csv", timesBCP, ',')
writedlm("data/BCPDiscountFixedStatusSmall.csv", statusBCP, ',')